In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import data as a dataframe
fitbit_merged  = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv')

In [ ]:
fitbit_merged.head()

In [ ]:
fitbit_merged.dtypes

In [ ]:
# display the correlation matrix
corr = fitbit_merged.corr()
corr.style.background_gradient(cmap = 'coolwarm')

In [ ]:
# only keep the last column
fitbit_corr = corr['Calories'].sort_values(ascending = False)
fitbit_corr

# Top 3 activities contribute to calories consumption are:
# 1. Distance
# 2. Very Active Minutes
# 3. Total Steps

In [ ]:
# join sleep time
# import data as a dataframe
fitbit_sleep  = pd.read_csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv')
fitbit_sleep.head()

In [ ]:
fitbit_sleep['SleepDay'] = pd.to_datetime(fitbit_sleep['SleepDay'])
fitbit_sleep.head()

In [ ]:
fitbit_merged['ActivityDate'] = pd.to_datetime(fitbit_merged['ActivityDate'])
fitbit_merged.head()

In [ ]:
# merge files
fitbit_sleep_merged = pd.merge(fitbit_merged, fitbit_sleep, left_on = ['Id', 'ActivityDate'], right_on = ['Id', 'SleepDay'], how = 'left')

In [ ]:
corr_2 = fitbit_sleep_merged.corr()
corr_2.style.background_gradient(cmap = 'coolwarm')
fitbit_corr_2 = corr_2['TotalMinutesAsleep'].sort_values(ascending = False)
fitbit_corr_2

# Top activities contribute to sleep time are:
# 1. Total time in bed
# 2. Lightly Active minutes

# Top activity negatively affects sleep time is: Sedentary minutes


# **Linear Regression**

In [ ]:
# check if there is any missing value
cols_with_missing = [col for col in fitbit_merged.columns
                    if fitbit_merged[col].isnull().any()]
cols_with_missing

In [ ]:
# Split the dataframe into train and test
from sklearn.model_selection import train_test_split

# separate X and y
y = fitbit_merged['Calories']
X = fitbit_merged.drop(['Calories', 'Id', 'ActivityDate'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# Perform linear regression
from sklearn.linear_model import LinearRegression
from sklearn import metrics

model = LinearRegression().fit(X_train, y_train)

print("R2: ", model.score(X_train, y_train))
print("\n Coef: ", model.coef_)


In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1 = df.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
# prediction with the model
y_pred = model.predict(X_test)

# Mean Squared Error (MSE)
print("Mean Squared Error: ", metrics.mean_squared_error(y_test, y_pred))

# Mean Absolute Error (MAE)
print("Mean Squared Error: ", metrics.mean_absolute_error(y_test, y_pred))

# Root Mean Squared Error (RMSE)
print("Mean Squared Error: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))